# Report: Solving the Cutting Stock Problem with Simulated Annealing

## Problem Description
Given a set of requested item lengths and the length of available stock materials, the objective is to find an optimal allocation that minimizes waste. The cutting stock problem can be mathematically formulated as an integer linear programming problem, where the decision variables represent the number of cuts made for each item.

## Algorithm: Simulated Annealing (SA)

The SA algorithm consists of the following steps:
1. Initialize the solution by allocating items(Greedy) to the stock materials.
2. Set an initial temperature and cooling rate.
3. Iterate until a stopping criterion is met (e.g., temperature or a target fitness value):
- **a**. Generate a neighboring solution by making a small modification to the current solution.
- **b**. Calculate the acceptance probability for the neighboring solution.
- **c**. If the neighboring solution is accepted, update the current solution.
- **d**. Decrease the temperature according to the cooling rate.
4. Output the best solution found during the iterations.

## Implementation Details
In our implementation, we have used the following components:
- **OpenFile:** This function reads the input file and extracts the stock length and requested item lengths.
- **Stock class:** This class represents a stock material and maintains a list of items allocated to it, as well as the remaining space in the stock material.
- **StockProb function:** This function calculates the probability of selecting a specific stock material for a given item based on the remaining space and item length.
- **generate_neighboring_solution function:** This function generates a neighboring solution by randomly selecting two stock materials and exchanging items between them.
- **checkRes function:** This function checks if all stock materials have non-negative space, indicating a valid solution.
- **acceptance_probability function:** This function calculates the probability of accepting a neighboring solution based on the difference in fitness scores between the current and neighboring solutions and the current temperature.
- **InitialStocks function:** This function initializes the stock materials by allocating requested items to them based on the StockProb function.


## Impact of Temperature
The temperature parameter in the simulated annealing algorithm has a significant impact on the search process. A higher temperature allows for more exploration and randomness in the search, preventing the algorithm from getting stuck in local minima. As the temperature decreases over iterations, the algorithm gradually shifts towards a more exploitative search, converging towards an optimal or near-optimal solution. The cooling of temperature balances the algorithm's exploration and exploitation tendencies, enabling it to efficiently navigate the solution space and avoid being trapped in suboptimal solutions.

## Can We Use a Linear Function for Temperature?
An important consideration in the simulated annealing algorithm is the choice of temperature function. While the commonly used cooling schedules, such as exponential or logarithmic, have proven effective, it is worth exploring alternative approaches, such as a linear function for temperature. A linear temperature function could offer a more gradual and steady decrease in temperature over iterations. However, it may also lead to premature convergence or insufficient exploration of the solution space. Further experimentation and analysis would be necessary to determine the suitability of a linear temperature function for the cutting stock problem and its impact on the algorithm's performance.


In [93]:
import math, random
from copy import deepcopy

In [94]:
def OpenFile(file):
    f=open(file).readlines()
    length=int(f[0].split()[2])
    requests=list(map(int, f[3].replace(' ','').split(',')))
    return length,requests

In [95]:
class Stock:
    def __init__(self,length) -> None:
        self.requests=[]
        self.space=length
    def setForRequest(self,length,i):
        self.requests.append(i)
        self.space-=length
    def fitness(self) -> int:
        return 0 if self.space>0 else -self.space
        
        

In [96]:
def StockProb(stock:Stock,request_len,length):
    if(stock.space-request_len<-length/10):
        return 0
    return length/5+stock.space
    

In [97]:
def calculateFitness(stocks):
    return sum([stock.fitness() for stock in stocks])

In [98]:
first,second =0,0

In [99]:
def generate_neighboring_solution(stocks,length,requests):
    res=deepcopy(stocks)
    try:
        first=random.choices(res,[0 if stock.space>0 else -stock.space for stock in res])[0]
        second=random.choices(res,[0 if stock.space<0 else stock.space for stock in res])[0]
    except:
        return stocks
    first:Stock
    second:Stock
    req1=random.choices(first.requests,[length+requests[i] for i in first.requests])[0]
    req2=random.choices(second.requests,[length-requests[i] for i in second.requests])[0]
    first.requests.append(req2)
    first.requests.remove(req1)
    first.space-=requests[req2]
    first.space+=requests[req1]
    second.requests.append(req1)
    second.requests.remove(req2)
    second.space-=requests[req1]
    second.space+=requests[req2] 
    return res

In [100]:
def checkRes(stocks):
    for stock in stocks:
        stock:Stock
        if(stock.space<0):
            return False
    return True

In [101]:
def acceptance_probability(stocks,neighbor,temperature):
    if(calculateFitness(stocks) > calculateFitness(neighbor)):
        return 1
    try:
        return math.exp(((-calculateFitness(stocks) + calculateFitness(neighbor)/temperature)))
    except:
        return 0
    

In [102]:
def InitialStocks(target,requests,length):
    stocks=[Stock(length) for _ in range(target)]
    for i in range(len(requests)):
        try:
            selected = random.choices(range(len(stocks)),[StockProb(stock,requests[i],length) for stock in stocks])[0]
        except:
            return InitialStocks(target,requests,length)
        stocks[selected].setForRequest(requests[i],i)
    return [stock for stock in stocks if stock.requests]

In [103]:
def SA(file,target,initial_temperature,cooling_rate):
    length,requests=OpenFile(file)
    stocks=InitialStocks(target,requests,length)
    temperature=initial_temperature
    while not checkRes(stocks) and temperature > 0 :
        for _ in range(10):
            neighbor=generate_neighboring_solution(stocks,length,requests)
            if(acceptance_probability(stocks,neighbor,temperature) > random.random()):
                stocks=neighbor
        temperature*=cooling_rate
    return stocks


In [104]:
res1=SA('./input1.stock',52,1000,0.99)

In [105]:
res2=SA('./input2.stock',90,1000,0.99)


In [106]:
res3=SA('./input3.stock',110,1000,0.99)

In [109]:
res4=SA('./input4.stock',230,1000,0.99)